In [1]:
#Sistema de recomendacion simple
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [2]:
#Recomendador basado en contenido
#Print plot overviews of the first 5 movies.
metadata['overview'].head()


0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [3]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(45466, 75827)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
def get_similarities_for_item(tfidf_matrix, item_index):
    from sklearn.metrics.pairwise import cosine_similarity
    from scipy.sparse import csr_matrix

    # Convertir tfidf_matrix a una matriz dispersa
    tfidf_matrix_sparse = csr_matrix(tfidf_matrix)

    # Obtener la fila del item en la matriz dispersa
    item_vector = tfidf_matrix_sparse[item_index]

    # Calcular la similitud solo para ese item
    cosine_sim = cosine_similarity(item_vector, tfidf_matrix_sparse)

    return cosine_sim

# Ejemplo: obtener similitudes para el primer item
similarities = get_similarities_for_item(tfidf_matrix, 0)


In [6]:
import pandas as pd

# Crear un dataframe para almacenar las similitudes
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

def get_recommendations_for_item(item_index, num_recommendations=10):
    # Obtener las similitudes para el item dado
    sim_scores = get_similarities_for_item(tfidf_matrix, item_index)

    # Convertir el array de similitud en una lista de pares (índice, similitud)
    sim_scores = list(enumerate(sim_scores[0]))

    # Ordenar las películas basadas en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares
    sim_scores = sim_scores[1:num_recommendations + 1]
    movie_indices = [i[0] for i in sim_scores]

    # Devolver las películas más similares
    return metadata['title'].iloc[movie_indices]

# Ejemplo: obtener recomendaciones para el primer item
recommendations = get_recommendations_for_item(0)
print(recommendations)


15348                                     Toy Story 3
2997                                      Toy Story 2
10301                          The 40 Year Old Virgin
24523                                       Small Fry
23843                     Andy Hardy's Blonde Trouble
29202                                      Hot Splash
43427                Andy Kaufman Plays Carnegie Hall
38476    Superstar: The Life and Times of Andy Warhol
42721    Andy Peters: Exclamation Mark Question Point
8327                                        The Champ
Name: title, dtype: object
